In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import bokeh 
import re
import spacy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.learning_curve import learning_curve
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

%matplotlib inline

In [4]:
depression=pd.read_csv("/Users/yonilevine/Desktop/Capstone/datasets/depressionnlp.csv",index_col=False)

In [5]:
depression['text'] = depression["text"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [6]:
cvt = CountVectorizer()
X = cvt.fit_transform(depression["text"])

In [7]:
X_df = pd.DataFrame(cvt.transform(depression['text']).todense(),
                       columns=cvt.get_feature_names())

word_counts = X_df.sum(axis=0)
print 'Number of words in corpus: ' + str(len(word_counts))
word_counts = word_counts[word_counts > 50]
print 'Number of words that occur more than fifty times: ' + str(len(word_counts)) + '\n'
print 'Least common: \n \n', word_counts.sort_values(ascending = False).tail(20), '\n'
print 'Most common: \n \n', word_counts.sort_values(ascending=False).head(20)

Number of words in corpus: 42773
Number of words that occur more than fifty times: 3569

Least common: 
 
flu            51
household      51
wherever       51
texas          51
television     51
technically    51
argued         51
associate      51
someones       51
cleaned        51
begins         51
sharp          51
blessed        51
scale          51
catching       51
receiving      51
rape           51
chores         51
christ         51
rope           51
dtype: int64 

Most common: 
 
to      165010
and     149728
the     108010
my       96925
it       72602
me       69100
of       68654
that     62042
in       52560
but      47614
have     47349
is       43860
for      42947
was      39667
just     38731
so       37967
this     36669
with     35865
like     30498
not      29126
dtype: int64


In [8]:
cvt = CountVectorizer(ngram_range=(2,6))
X_all = cvt.fit_transform(depression["text"])
X_all_sum = np.sum(X_all, axis=0)
word_counts = [(word, X_all_sum[0, index])
              for index, word in enumerate(cvt.get_feature_names())]
word_counts.sort(key=lambda x: x[1], reverse=True)

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
depression_train, depression_test = train_test_split(depression, 
                                               stratify=depression.insuicide,
                                               test_size=.3, 
                                               random_state=9245)

In [ ]:
depression_train.shape

In [ ]:
depression_test.shape

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('cls', MultinomialNB())]) 
pipeline.fit(depression_train['text'], depression_train['insuicide'])


pipeline.score(depression_test['text'], depression_test['insuicide'])

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('cls', BernoulliNB())]) 
pipeline.fit(depression_train['text'], depression_train['insuicide'])

pipeline.score(depression_test['text'], depression_test['insuicide'])

In [ ]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('cls', LogisticRegression())]) 
pipeline.fit(depression_train['text'], depression_train['insuicide'])

pipeline.score(depression_test['text'], depression_test['insuicide'])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('cls', RandomForestClassifier(n_estimators=200, random_state=9245))])
    
pipeline.fit(depression_train['text'], depression_train['insuicide'])


pipeline.score(depression_test['text'], depression_test['insuicide'],)


In [ ]:
depression.insuicide.sum()

In [ ]:
len(depression)

In [ ]:
8537/17397.0

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
params={'penalty':['l1','l2'], 'C':[.01,.05,.1,.5,1.0,5,10,20]}

In [ ]:
lr=LogisticRegression()
lrgrid=GridSearchCV(LogisticRegression(),param_grid=params,verbose=1,cv=5)

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('gridsearch',lrgrid),
    ('logistic_regression', LogisticRegression())
    ]) 
pipeline.fit(depression_train['text'], depression_train['insuicide'])

pipeline.score(depression_test['text'], depression_test['insuicide'])